---
### Import Libraries
---

In [1]:
from flask import Blueprint, request, jsonify
from app import db
from models import Task, TaskSchema, StatusEnum, PriorityEnum

ImportError: cannot import name 'TaskSchema' from 'models' (C:\Users\voudo\OneDrive\Υπολογιστής\THECUBE_Python Developer Technical Assessment\models.py)

---
### routes
---

In [ ]:
tasks_bp = Blueprint("tasks_bp", __name__)
task_schema = TaskSchema()
tasks_schema = TaskSchema(many=True)

@tasks_bp.route("/", methods=["GET"])
def get_tasks():
    status = request.args.get("status")
    priority = request.args.get("priority")
    sort_by = request.args.get("sort_by", "created_at")
    order = request.args.get("order", "asc")

    query = Task.query

    if status:
        query = query.filter_by(status=status)
    if priority:
        query = query.filter_by(priority=priority)

    if order == "desc":
        query = query.order_by(db.desc(getattr(Task, sort_by)))
    else:
        query = query.order_by(db.asc(getattr(Task, sort_by)))

    tasks = query.all()
    return jsonify(tasks_schema.dump(tasks)), 200


@tasks_bp.route("/<int:id>", methods=["GET"])
def get_task(id):
    task = Task.query.get(id)
    if not task:
        return jsonify({"error": "Task not found"}), 404
    return task_schema.jsonify(task)


@tasks_bp.route("/", methods=["POST"])
def create_task():
    data = request.get_json()
    if not data or "title" not in data:
        return jsonify({"error": "Title is required"}), 400

    new_task = Task(
        title=data["title"],
        description=data.get("description", ""),
        status=data.get("status", "todo"),
        priority=data.get("priority", "medium"),
    )

    db.session.add(new_task)
    db.session.commit()

    return task_schema.jsonify(new_task), 201


@tasks_bp.route("/<int:id>", methods=["PUT"])
def update_task(id):
    task = Task.query.get(id)
    if not task:
        return jsonify({"error": "Task not found"}), 404

    data = request.get_json()

    for field in ["title", "description", "status", "priority"]:
        if field in data:
            setattr(task, field, data[field])

    db.session.commit()
    return task_schema.jsonify(task), 200


@tasks_bp.route("/<int:id>", methods=["DELETE"])
def delete_task(id):
    task = Task.query.get(id)
    if not task:
        return jsonify({"error": "Task not found"}), 404

    db.session.delete(task)
    db.session.commit()
    return jsonify({"message": "Task deleted successfully"}), 200